## D'NOVA YouTube Shorts SCraping Flow
-  Test code: 테스트 코드
- Daily Shorts check: 수집 데이터 loading
#### Code Flow
1. 기존 수집했던 쇼츠 재수집 -7d, -14d, -21d
2. channel shorts url input으로 들어갈시 채널 url 필터링(daily)
3. 수집한 channel list 수집 후 loaded_db에 적재 

### Test Code

In [145]:
import requests
import re
import numpy as np
from bs4 import BeautifulSoup

# Define the payload and headers for the request
# shorts channel1: https://www.youtube.com/watch?v=xEuH_5Ik92U
# shorts channel2: https://www.youtube.com/watch?v=fa_sSaHxTZ4
# yt_shorts page: https://www.youtube.com/@%EC%95%84%EC%9D%B4%EB%8F%8C/shorts
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
payload = {
    'api_key': API_KEY,
    'url': 'https://www.youtube.com/watch?v=fa_sSaHxTZ4'
    ,'render': 'true'
}
# payload = { 'api_key': '3397e8a6a37e89b03b08750a27575df8', 'url': 'https://www.youtube.com/watch?v=xEuH_5Ik92U' }
# r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
# print(r.text)
headers = {
    'x-sapi-render': 'true',
    'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 3}]'
    # , {"type": "click", "selector": {"type": "text","value": "Show transcript"}}
    # ,{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2} 
}
# Make the GET request
r_full = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False) # headers=headers,

# Save the response text to a file
with open('response_channel_test.txt', 'w', encoding='utf-8') as file:
    file.write(r_full.text)

print("Response saved to response_output.txt")


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 

In [77]:
# url 한개만 추출 -> comments (x)
import requests

payload = { 'api_key': '3397e8a6a37e89b03b08750a27575df8', 'url': 'https://www.youtube.com/watch?v=xEuH_5Ik92U'}
r_normal = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
# Save the response text to a file
with open('response_channel_test_쇼츠페이지.txt', 'w', encoding='utf-8') as file:
    file.write(r_normal.text)

print("Response saved to response_output.txt")

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response saved to response_output.txt


In [112]:
# url 여러개 진행
import requests
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
# List of URLs to test
urls = [
    'https://www.youtube.com/watch?v=xEuH_5Ik92U',
    'https://www.youtube.com/watch?v=CPsCEfs3m8I',
    'https://www.youtube.com/watch?v=jXUHFbNo5Vg'
]

# API key and headers
headers = {
    'x-sapi-render': 'true',
    # 'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
    'x-sapi-instruction_set': '[{"type":"loop","for":' + str(len(urls))+',"instructions":[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]}]'
}

# Loop through each URL, send request, and save response to a file
for i, url in enumerate(urls):
    payload = {
        'api_key': API_KEY,
        'url': url,
        'render': 'true'
    }
    
    # Send the request
    r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
    
    # Save the response to a file, naming it based on the URL index
    filename = f'response_{i + 1}.txt'
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(r.text)
    
    print(f'Response for {url} saved to {filename}')



c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=xEuH_5Ik92U saved to response_1.txt


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=CPsCEfs3m8I saved to response_2.txt


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Response for https://www.youtube.com/watch?v=jXUHFbNo5Vg saved to response_3.txt


### Daily Shorts Check
- 95000 채널 매일 확인 KST 23:30 진행 -> request 95000 기본
- channel_df: YT 채널 데이터
- loaded_df: 적제 쇼츠 데이터
- basic_shorts_info: 기본적인 Shorts 데이터 추출

In [147]:
# 기존 channel list와 적재 데이터 loaded
import requests
import pandas as pd
from datetime import datetime, timedelta
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
# 한국, 미국 데이터 9507개
channel_df = pd.read_excel('us_korea_test.xlsx')[:10]
# 적재 DB: # -7d, -14d, -21d 만 필터링
loaded_df = pd.read_excel("loaded_db.xlsx")
today = datetime.today()
try:
    loaded_df['check_date'] = loaded_df['check_date'].dt.date
    filter_dates = [today.date() - timedelta(days=7), today.date() - timedelta(days=14), today.date() - timedelta(days=21)]
    # Filter the dataframe based on check_date
    loaded_df = loaded_df[loaded_df['check_date'].isin(filter_dates)]
    # channel_df 와 loaded_df 교집합 데이터
    merged_df = pd.merge(loaded_df, channel_df, on='channel_name', how='inner')
except Exception:
    pass

print(loaded_df)

Empty DataFrame
Columns: [channel_name, shorts_url, shorts_title, shorts_description, shorts_thumbnail, shorts_view, shorts_likes, shorts_comments_num, shorts_comments_time, shorts_published_date, check_date]
Index: []


In [148]:
# Test 용
# r_full: render 조건 True, request: 10, 쇼츠 데이터
# r_normal: render 조건 False, request: 1, 쇼츠데이터
# r_page: 채널 쇼츠 페이지, reqeust: 1

# scraperAPI result filter with NaN values for missing data
def basic_shorts_info(text):
    # shorts title
    match_title = re.search(r'<title>(.*?) - YouTube</title>', text)
    extracted_title = match_title.group(1) if match_title else np.nan

    # shorts description
    description_pattern = r'attributedDescription":{"content":"(.*?)"'
    # Find the match
    description_match = re.search(description_pattern, text)
    # Extract the content if found
    extracted_description = description_match.group(1) if description_match else None
    
    # shorts total view (date and time)
    date_time_pattern = r'datePublished" content="(\d{4}-\d{2}-\d{2})T(\d{2}):'
    match_date = re.search(date_time_pattern, text)
    extracted_published = f"{match_date.group(1)} {match_date.group(2)}" if match_date else np.nan

    # Thumbnail
    thumbnail_pattern = r'"url":"(https?://[^\s"]+)"'
    thumbnail_match = re.search(thumbnail_pattern, text)
    yt_shorts_thumbnail = thumbnail_match.group(1) if thumbnail_match else np.nan

    # Total view
    view_pattern = r'"viewCountEntityKey":"[^"]+","factoid":\{"factoidRenderer":\{"value":\{"simpleText":"([\d,]+)"'
    view_match = re.search(view_pattern, text)
    extracted_view = view_match.group(1) if view_match else np.nan

    # Total likes
    like_pattern = r'"iconName":"LIKE","title":"([\d.]+)([KM]?)"'
    like_match = re.search(like_pattern, text)
    if like_match:
        extracted_likes = float(like_match.group(1))
        suffix = like_match.group(2)
        if suffix == 'K':
            extracted_likes *= 1000
        elif suffix == 'M':
            extracted_likes *= 1000000
    else:
        extracted_likes = np.nan

    # Total comments    
    comment_num_pattern = r'"text":"Comments"}\]},"contextualInfo":\{"runs":\[\{"text":"([\d.]+)([KM]?)"'
    comment_num_match = re.search(comment_num_pattern, text)
    if comment_num_match:
        extracted_comments_number = float(comment_num_match.group(1))
        suffix = comment_num_match.group(2)
        if suffix == 'K':
            extracted_comments_number *= 1000
        elif suffix == 'M':
            extracted_comments_number *= 1000000
    else:
        extracted_comments_number = np.nan

    # yt shorts comment 결과 및 comment 작성 시간
    comments_time_text_pattern = r'<span[^>]*id="published-time-text"[^>]*>\s*<a[^>]*>\s*(.*?)\s*<\/a>.*?<yt-attributed-string[^>]*>\s*<span[^>]*class="yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap"[^>]*>\s*(.*?)\s*<\/span>'
    
    # Find all matches for time and comments
    comments_time_matches = re.findall(comments_time_text_pattern, text, re.DOTALL)
    
    # Process and clean comments
    extracted_comments_time_matches = []
    if comments_time_matches:
        for time, comment in comments_time_matches:
            # Clean the comment to remove unwanted tags using BeautifulSoup
            soup = BeautifulSoup(comment, 'html.parser')
            cleaned_comment = soup.get_text()
            extracted_comments_time_matches.append((time.strip(), cleaned_comment.strip()))
    else:
        extracted_comments_time_matches = np.nan

    return (extracted_title, extracted_description, extracted_published, yt_shorts_thumbnail, 
            extracted_view, extracted_likes, extracted_comments_number, extracted_comments_time_matches)
yt_shorts_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r_full.text)
print("yt_shorts_title: ", yt_shorts_title)
print("yt_shorts_description: ", yt_shorts_description)
print("yt_shorts_published: ", yt_shorts_published)
print("yt_shorts_thumbnalil: ", yt_shorts_thumbnail)
print("yt_view: ", yt_shorts_view)
print("yt_likes: ", yt_shorts_likes)
print("yt_comments: ", yt_shorts_comments_num)
print("yt_shorts_comments_time: ", yt_shorts_comments_time)
# print(len(yt_shorts_comments_time))

yt_shorts_title:  Just Trust God
yt_shorts_description:  None
yt_shorts_published:  2022-10-21 16
yt_shorts_thumbnalil:  https://rr4---sn-bpb5oxu-3c2y.googlevideo.com/videoplayback?expire=1729256387\u0026ei=YwcSZ_W4JbCli9oP3Mus0QY\u0026ip=104.250.200.157\u0026id=o-AIqq3p-zL54fhxpGWLpYfSLW2HNez7CmN328RmdwkOUE\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1729234787%2C\u0026mh=31\u0026mm=31%2C29\u0026mn=sn-bpb5oxu-3c2y%2Csn-8vap5-3c2d\u0026ms=au%2Crdu\u0026mv=m\u0026mvi=4\u0026pl=22\u0026rms=au%2Cau\u0026pcm2=yes\u0026initcwndbps=2145000\u0026bui=AXLXGFRnKHAWIVMwxsC0BrUOn4mkYavtEXe2Jl7hi1xde5GjeaHdfnVfoUV78GYboMvTSXoRo4Ipk5e7\u0026spc=54MbxYERAZtMh9LMA3ysYSYOnRjF65dPgpZAUxV0WletwsYEDOH7TgAsgDl2zS4\u0026vprv=1\u0026svpuc=1\u0026mime=video%2Fmp4\u0026ns=MiHLoWAOwuVBwlwJkLdAqHwQ\u0026rqh=1\u0026cnr=14\u0026ratebypass=yes\u0026dur=54.613\u0026lmt=1700141138840856\u0026mt=1729234388\u0026fvip=2\u0026fexp=51312688\u0026c=WEB\u0026sefc=1\u0026txp=631022

##### 1. 기존 수집했던 쇼츠 재수집 -7d, -14d, -21d
- loaded_db에서 날짜 필터링해서 데이터 수집 진행

In [149]:
import requests
from datetime import datetime, timedelta

def collect_past_shorts(api_key, loaded_df):
    today = datetime.today()
    print(loaded_df)
    # Iterate over each row in loaded_df, checking check_date and shorts_url
    for idx, (check_dt, shorts_url) in enumerate(zip(loaded_df['check_date'], loaded_df['shorts_url'])):
        if check_dt == (today.date() - timedelta(days=14)) or check_dt == (today.date() - timedelta(days=21)):
            payload = {
                'api_key': api_key,
                'url': shorts_url
            }
            
            # Send the request
            r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)

            # Extract relevant information from the response
            yt_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r.text)
            # Prepare new row with updated values (keeping the same channel name and other values from loaded_df)
            new_row = {
                'channel_nm': loaded_df.loc[idx, 'channel_nm'], 
                'shorts_url': shorts_url, 
                'shorts_title': yt_title,
                'shorts_description': yt_shorts_description,
                'shorts_thumbnalil': yt_shorts_thumbnail,
                'shorts_view': yt_shorts_view,
                'shorts_likes': yt_shorts_likes,
                'shorts_comments': yt_shorts_comments_num,
                'shorts_comments_time': yt_shorts_comments_time,
                'shorts_published_date': yt_shorts_published,
                'check_date': today.date()  # Use today's date for the new check_date
            }
            
            # Append the new row to the DataFrame
            loaded_df = loaded_df.append(new_row, ignore_index=True)

        # -7d에는 comments 추가 수집
        elif check_dt==(today.date() - timedelta(days=7)): 
            payload = {
                'api_key': API_KEY,
                'url': 'https://www.youtube.com/watch?v=xEuH_5Ik92U',
                'render': 'true'
            }

            headers = {
                'x-sapi-render': 'true',
                'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2}]'
                # , {"type": "click", "selector": {"type": "text","value": "Show transcript"}}
                # ,{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 2} 
            }
            r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
            yt_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r.text)

            # Prepare new row with updated values (keeping the same channel name and other values from loaded_df)
            new_row = {
                'channel_nm': loaded_df.loc[idx, 'channel_nm'], 
                'shorts_url': shorts_url, 
                'shorts_title': yt_title,
                'shorts_description': yt_shorts_description,
                'shorts_thumbnalil': yt_shorts_thumbnail,
                'shorts_view': yt_shorts_view,
                'shorts_likes': yt_shorts_likes,
                'shorts_comments': yt_shorts_comments_num,
                'shorts_comments_time': yt_shorts_comments_time,
                'shorts_published_date': yt_shorts_published,
                'check_date': today.date()  # Use today's date for the new check_date
            }

            loaded_df = loaded_df.append(new_row, ignore_index=True)

        else:
            print("no data")
            pass

    return loaded_df

collect_past_shorts(API_KEY, loaded_df)

Empty DataFrame
Columns: [channel_name, shorts_url, shorts_title, shorts_description, shorts_thumbnail, shorts_view, shorts_likes, shorts_comments_num, shorts_comments_time, shorts_published_date, check_date]
Index: []


,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date


##### 2. channel shorts url input으로 들어갈시 채널 url 필터링(daily)
- 최신 기준으로 맵핑이 되지 않는 경우 최근 1개 url수집 
- 기존 말고 신규 shorts url 확인시 list 추가
- 마지막에 다 확인후 추가된 list 쇼츠 데이터 수집 진행 -> 10 request로

In [150]:
# 매일 전체 채널 필터링
# - 신규 쇼츠 데이터 발견시 list에 추가해서 comments까지 확인
# - 과거 데이터가 없을 수 있으니 만약 하나도 매칭이 안되면 3개 수집, 있다면 그거 앞에꺼를 추가수집
def scanning_yt_shorts_page(api_key, channel_db, loaded_db):
    # List to store results in the format (channel_name, new_shorts_url)
    new_shorts_urls = []

    # Iterate over each row in channel_db
    for index, row in channel_db.iterrows():
        channel_name = row['channel_name']
        yt_fixed_url = row['yt_fixed_url']
        
        # Filter loaded_db where channel_name matches
        matching_rows = loaded_db[loaded_db['channel_name'] == channel_name]
        
        if not matching_rows.empty:
            # Sort by check_date to get the most recent row
            latest_row = matching_rows.sort_values(by='check_date', ascending=False).iloc[0]
        
        # Prepare the payload for ScraperAPI request, using yt_fixed_url
        payload = {
            'api_key': api_key,
            'url': yt_fixed_url
        }
        
        # Send the request to ScraperAPI
        r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
        
        # Define the regex pattern to extract Shorts URLs
        shorts_url_pattern = r'webCommandMetadata":{"url":"(/shorts/\w+)"'
        # Find all matching Shorts URLs
        shorts_urls = re.findall(shorts_url_pattern, r.text)
        shorts_urls =  ['https://www.youtube.com' + url.replace('/shorts/', '/watch?v=') for url in shorts_urls]

        print(f"shorts_urls for {channel_name}: ", shorts_urls)

        # Check if there are any new shorts URLs before the latest one
        if not matching_rows.empty:
            latest_url = latest_row['shorts_url']
            # If the latest URL exists in the list of Shorts URLs, find the ones before it
            if latest_url in shorts_urls:
                matching_index = shorts_urls.index(latest_url)
                # Collect all URLs before the match with channel_name
                for shorts_url in shorts_urls[:matching_index]:
                    new_shorts_urls.append((channel_name, shorts_url))
        else:
            # If no match in loaded_db, return the first shorts URL with channel_name
            if shorts_urls:
                new_shorts_urls.append((channel_name, shorts_urls[0]))
    

    return new_shorts_urls  # Return the list of tuples (channel_name, shorts_url)

In [151]:
new_shorts_urls = scanning_yt_shorts_page('3397e8a6a37e89b03b08750a27575df8', channel_df, loaded_df)
print(new_shorts_urls)

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Robert Breaker:  ['https://www.youtube.com/watch?v=qc0iqmlAR38', 'https://www.youtube.com/watch?v=Lu2gWsx3rs8', 'https://www.youtube.com/watch?v=KqpYUbvjRK8', 'https://www.youtube.com/watch?v=HQIiA6T_Iic', 'https://www.youtube.com/watch?v=brm8V4CEnh8', 'https://www.youtube.com/watch?v=4qyiflpsY5E', 'https://www.youtube.com/watch?v=nz6UeE75hOM', 'https://www.youtube.com/watch?v=Vcw2rL2nWfA', 'https://www.youtube.com/watch?v=5ndfKljP5gA', 'https://www.youtube.com/watch?v=c0_YNvm5sJw', 'https://www.youtube.com/watch?v=zwUMeX3BPbU', 'https://www.youtube.com/watch?v=6XcJDpLrqr8', 'https://www.youtube.com/watch?v=2OIxmnYPc3s', 'https://www.youtube.com/watch?v=kEWh0IZyPiE', 'https://www.youtube.com/watch?v=vBonWvTDeyM', 'https://www.youtube.com/watch?v=VVUCLE0bths', 'https://www.youtube.com/watch?v=0PSQuoCVELg', 'https://www.youtube.com/watch?v=zzN6n_GJHtA', 'https://www.youtube.com/watch?v=C7eyrX2WjOs', 'https://www.youtube.com/watch?v=tkeh2Oz29Gw', 'https://www.youtube.com/w

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Nick Jones:  ['https://www.youtube.com/watch?v=fa_sSaHxTZ4', 'https://www.youtube.com/watch?v=0vPxMeWXNpA', 'https://www.youtube.com/watch?v=mlzNZAnY2Vc', 'https://www.youtube.com/watch?v=4LHlp1eCQIE', 'https://www.youtube.com/watch?v=wWHyYFemnWk', 'https://www.youtube.com/watch?v=LZiy319wB0Q', 'https://www.youtube.com/watch?v=sQi9mAxFAC0', 'https://www.youtube.com/watch?v=OkBVU_zFNh4', 'https://www.youtube.com/watch?v=OnDZ79Jd4iU', 'https://www.youtube.com/watch?v=P3itqoXsh1M', 'https://www.youtube.com/watch?v=t0AIf6QNd9Y', 'https://www.youtube.com/watch?v=7UmYhoddWo0', 'https://www.youtube.com/watch?v=twLOUozjmzU', 'https://www.youtube.com/watch?v=Twljy9DtxB4', 'https://www.youtube.com/watch?v=m70O1UpiuMY', 'https://www.youtube.com/watch?v=up0TKqllXJ4', 'https://www.youtube.com/watch?v=66xZITjBlqA', 'https://www.youtube.com/watch?v=0zKSBOIVR6c', 'https://www.youtube.com/watch?v=9x0aAFCHrxA', 'https://www.youtube.com/watch?v=yQy8NMII_Ns', 'https://www.youtube.com/watch

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

shorts_urls for AoC Network:  []


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for ALIF FUN TIME:  ['https://www.youtube.com/watch?v=iI39fwgM0wI', 'https://www.youtube.com/watch?v=bJBqOsOBQVc', 'https://www.youtube.com/watch?v=fAn2dQQTzv8', 'https://www.youtube.com/watch?v=brLUjKlD0RQ', 'https://www.youtube.com/watch?v=Fv8iy1hE9Rk', 'https://www.youtube.com/watch?v=s6hBiLycPGU', 'https://www.youtube.com/watch?v=bUYSmEvkkpQ', 'https://www.youtube.com/watch?v=w3ESLEPufzI', 'https://www.youtube.com/watch?v=p2NjIUUPRPM', 'https://www.youtube.com/watch?v=AOyqnoGZdF0', 'https://www.youtube.com/watch?v=qYyCNhr_ktg', 'https://www.youtube.com/watch?v=ylQv4C0qxmI', 'https://www.youtube.com/watch?v=SLq86dAZcWA', 'https://www.youtube.com/watch?v=QkQdk8RLnQ8', 'https://www.youtube.com/watch?v=mMN0zIr1RGc', 'https://www.youtube.com/watch?v=yt3kDwzTl3Y', 'https://www.youtube.com/watch?v=YgFdVp0Exm0', 'https://www.youtube.com/watch?v=O_oHt1Otq4I', 'https://www.youtube.com/watch?v=E3gpt5fj6y4', 'https://www.youtube.com/watch?v=4FzbCqPSu2w', 'https://www.youtube.com/wa

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


shorts_urls for Genetically Modified Skeptic:  ['https://www.youtube.com/watch?v=IzQDQrpPZ38', 'https://www.youtube.com/watch?v=VXNgLpo_Ig0', 'https://www.youtube.com/watch?v=12ZbieyImPs', 'https://www.youtube.com/watch?v=hLBMTwJo1zQ', 'https://www.youtube.com/watch?v=jIfV1mKWQzY', 'https://www.youtube.com/watch?v=eZYdVU7NJvs', 'https://www.youtube.com/watch?v=OAlVVmQBCS0', 'https://www.youtube.com/watch?v=X3bA5t3E3RE', 'https://www.youtube.com/watch?v=Ilt7xwIaTFQ', 'https://www.youtube.com/watch?v=egX2fYNAiGc', 'https://www.youtube.com/watch?v=BtQf5XU3fLU', 'https://www.youtube.com/watch?v=Ew_gBgLlVvs', 'https://www.youtube.com/watch?v=jFdgYsth32o', 'https://www.youtube.com/watch?v=kFmQ31j_blM', 'https://www.youtube.com/watch?v=aTLOUoUB08A']
[('Robert Breaker', 'https://www.youtube.com/watch?v=qc0iqmlAR38'), ('Nick Jones', 'https://www.youtube.com/watch?v=fa_sSaHxTZ4'), ('Steve Ram', 'https://www.youtube.com/watch?v=SqU7aeAPkrY'), ('So Sweet ASMR', 'https://www.youtube.com/watch?v=k27

##### 3. 수집한 channel list 수집 후 loaded_db에 적재
- new_shorts_urls 기준으로 rendered=True로 수집
- 수집 데이터 loaded_db에 적재 후 저장

In [152]:
# url 여러개 진행
import requests
API_KEY = '3397e8a6a37e89b03b08750a27575df8'
today = datetime.today()
def add_new_shorts_data(api_key, urls):
    data_list = []
    check_date = today
    # Separate channel names and shorts URLs
    channel_names = [item[0] for item in urls]
    shorts_urls = [item[1] for item in urls]
    
    for channel_name, url in zip(channel_names, shorts_urls):
        print(channel_name, url)
        shorts_url = url
        # Prepare payload and headers for the request
        payload = {
            'api_key': api_key,
            'url': url,
            'render': 'true'
        }

        headers = {
            'x-sapi-render': 'true',
            'x-sapi-instruction_set': '[{"type":"scroll","direction":"y","value":"bottom"}, {"type": "wait", "value": 3}]'
        }

        # Make the GET request
        r = requests.get('https://api.scraperapi.com/', params=payload, headers=headers, verify=False)
        print(r.text[:20])

        # Extract the necessary data
        shorts_title, shorts_description, shorts_published, shorts_thumbnail, shorts_view, shorts_likes, shorts_comments_num, shorts_comments_time = basic_shorts_info(r.text)
        
        # Print the extracted information for debugging
        print("title: ", channel_name)
        print("shorts_title: ", shorts_title)
        print("shorts_description: ", shorts_description)
        print("shorts_published: ", shorts_published)
        print("shorts_thumbnalil: ", shorts_thumbnail)
        print("view: ", shorts_view)
        print("likes: ", shorts_likes)
        print("comments: ", shorts_comments_num)
        print("shorts_comments_time: ", shorts_comments_time)

        # Append as a tuple to the data_list
        data_list.append((
            channel_name,
            shorts_url, 
            shorts_title,  
            shorts_description, 
            shorts_thumbnail, 
            shorts_view, 
            shorts_likes, 
            shorts_comments_num, 
            shorts_comments_time,
            shorts_published, 
            check_date

        ))
    
    return data_list

data_list = add_new_shorts_data(API_KEY, new_shorts_urls)
print(data_list)



Robert Breaker https://www.youtube.com/watch?v=qc0iqmlAR38


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<!DOCTYPE html><html
title:  Robert Breaker
shorts_title:  Watching Plane Take Off From a Plane #airtravel #airplane #airport t
shorts_description:  None
shorts_published:  nan
shorts_thumbnalil:  https://support.google.com/youtube/answer/3037019#zippy=%2Ccheck-that-youre-signed-into-youtube
view:  6,045
likes:  252.0
comments:  16.0
shorts_comments_time:  [('5 days ago', 'I never get tired of watching planes, taking off or landing! Still one of man’s most beautiful inventions!'), ('5 days ago', 'Thank you for sharing, brother Breaker'), ('2 days ago', 'You one of the things that I have learn is when God gives us eternal life he will never take it back because if he did then it would not be eternal life even if we mess up but God will correct us as his children but he will never take away eternal life from us'), ('4 days ago', 'Just like the airplane is ready to take off it reminds me of the rapture of the church that will take place real soon'), ('5 days ago', 'Ret Captain. If this is

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<!DOCTYPE html><html
title:  Nick Jones
shorts_title:  Just Trust God
shorts_description:  None
shorts_published:  2022-10-21 16
shorts_thumbnalil:  https://rr1---sn-f58xn2xxq-aj5l.googlevideo.com/videoplayback?expire=1729259765\u0026ei=lRQSZ4r3LL-dp-oP1K2igAk\u0026ip=45.131.94.92\u0026id=o-AN3AqzGvD8nlNDug7W3ckZzNbUrmIFiPwJSjp5FVjNte\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1729238165%2C\u0026mh=31\u0026mm=31%2C29\u0026mn=sn-f58xn2xxq-aj5l%2Csn-hgn7rn7r\u0026ms=au%2Crdu\u0026mv=u\u0026mvi=1\u0026pl=24\u0026rms=au%2Cau\u0026bui=AXLXGFSFd4wawj8t4AkOZBaXmbuwAeh7HzTA5Ws7LnCGCK-F-1ic-Qj4Fs95J6O_oA3DkFbd2xJ0caaE\u0026spc=54MbxYgGcuehe_D4-eVPEbzTaYiOm9-KciZWxcCERHvQngr-O45gB0LGyW6MduE\u0026vprv=1\u0026svpuc=1\u0026mime=video%2Fmp4\u0026ns=VhPh7QVcAVU55_70Yx75bwwQ\u0026rqh=1\u0026cnr=14\u0026ratebypass=yes\u0026dur=54.613\u0026lmt=1700141138840856\u0026mt=1729236892\u0026fvip=5\u0026fexp=51312688\u0026c=WEB\u0026sefc=1\u0026txp=6310224\u0026n=sTd

...

<!DOCTYPE html><html
title:  Web Inglés
shorts_title:  Intensificadores en Inglés #shorts
shorts_description:  Adverbios intensificadores ¿Qué tan cerca está el león? Aprende describir distancia.
shorts_published:  2022-12-17 14
shorts_thumbnalil:  https://rr3---sn-5ualdns6.googlevideo.com/videoplayback?expire=1729259881\u0026ei=CRUSZ4-tB4OCkucPwdHJ4QM\u0026ip=2606%3A97c0%3Aab32%3Ae39d%3Aaa5c%3Acfdd%3A1a6a%3A6a80\u0026id=o-ALlIf3NWdctUqGtnEhym7V8YN1qteIwpT5vChmzb_QKN\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1729238281%2C\u0026mh=sU\u0026mm=31%2C29\u0026mn=sn-5ualdns6%2Csn-5uaeznze\u0026ms=au%2Crdu\u0026mv=m\u0026mvi=3\u0026pl=32\u0026rms=au%2Cau\u0026initcwndbps=1013750\u0026bui=AXLXGFSXGYl6eOiHU04Z4SQoN_QeXadjXv9UD-_h8kPGs85Lsqoxy1VBcQP2gbNOSj9JwMVtEjJls3cT\u0026spc=54MbxWYD_CAts9OHXyfv1z1yzym00-toWm7K1Qg1nUdpCid1krb-NSKmHNRYY2g\u0026vprv=1\u0026svpuc=1\u0026mime=video%2Fmp4\u0026ns=2a0RSUvQIB9i_8f1Xh_uVbgQ\u0026rqh=1\u0026gir=yes\u0026cl

c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<!DOCTYPE html><html
title:  ALIF FUN TIME
shorts_title:  nan
shorts_description:  None
shorts_published:  nan
shorts_thumbnalil:  nan
view:  nan
likes:  nan
comments:  nan
shorts_comments_time:  nan
Genetically Modified Skeptic https://www.youtube.com/watch?v=IzQDQrpPZ38


c:\Users\T14Gen1\Documents\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<!DOCTYPE html><html
title:  Genetically Modified Skeptic
shorts_title:  Gen Z is MORE Muslim than Boomers here?
shorts_description:  Gen Z is more Muslim, Buddhist, and Hindu than Boomers in some countries. Generation Z is less Christian than previous generations, but not always less religious.\n\nClip from my video: Surprising Statistics About Gen Z's Religious Views
shorts_published:  2023-12-10 08
shorts_thumbnalil:  https://rr5---sn-h0jelne7.googlevideo.com/videoplayback?expire=1729259921\u0026ei=MRUSZ_ePJJi56dsPtPaJ-Qo\u0026ip=84.33.200.111\u0026id=o-AI0o3Rx6sWkanouPJ3iD-LHVk1UIKOO8qlXuKPlfb2tc\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1729238321%2C\u0026mh=Oc\u0026mm=31%2C26\u0026mn=sn-h0jelne7%2Csn-4g5e6nss\u0026ms=au%2Conr\u0026mv=m\u0026mvi=5\u0026pl=19\u0026rms=au%2Cau\u0026initcwndbps=4206250\u0026bui=AXLXGFQs2FiNdazlw90k9Cv-XiBSv9U0q9aD-wPJAlqweJV0u2ZafSiCBfpwCChfyo9ilIgm-wfo11Pl\u0026spc=54MbxVgNx3ZtqhmhAxiq8O9FanS7GRKQKvfJAmN

In [153]:
# 추가된 데이터 dataframe으로 loaed_db에 추가
def add_new_data_to_loaded_db(data_list, loaded_db):
    # Define the columns for the DataFrame
    columns = [
        'channel_name', 'shorts_url', 'shorts_title', 'shorts_description', 
        'shorts_thumbnail', 'shorts_view', 'shorts_likes', 
        'shorts_comments_num', 'shorts_comments_time', 
        'shorts_published_date', 'check_date'
    ]
    
    # Map the data_list to the correct columns and create a new DataFrame
    new_data_df = pd.DataFrame(data_list, columns=columns)
    
    # Append the new data to the loaded_db DataFrame
    updated_loaded_db = pd.concat([loaded_db, new_data_df], ignore_index=True)
    
    return updated_loaded_db

updated_loaded_db = add_new_data_to_loaded_db(data_list, loaded_df)
updated_loaded_db

C:\Users\T14Gen1\AppData\Local\Temp\ipykernel_9440\1338605675.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_loaded_db = pd.concat([loaded_db, new_data_df], ignore_index=True)


,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date
0,Robert Breaker,https://www.youtube.com/watch?v=qc0iqmlAR38,Watching Plane Take Off From a Plane #airtrave...,None,https://support.google.com/youtube/answer/3037...,"6,045",252.0,16.0,"[(5 days ago, I never get tired of watching pl...",NaN,2024-10-18 16:55:26.581137
1,Nick Jones,https://www.youtube.com/watch?v=fa_sSaHxTZ4,Just Trust God,None,https://rr1---sn-f58xn2xxq-aj5l.googlevideo.co...,"90,878",17000.0,552.0,NaN,2022-10-21 16,2024-10-18 16:55:26.581137
2,Steve Ram,https://www.youtube.com/watch?v=SqU7aeAPkrY,Will You Comply? New Coming Mandates in 2023,FULL VIDEO • TRUMP BLASTS Bringing Back COV...,https://rr5---sn-5uh5o-f5f6.googlevideo.com/vi...,"56,653",3200.0,504.0,NaN,2023-09-04 15,2024-10-18 16:55:26.581137
3,So Sweet ASMR,https://www.youtube.com/watch?v=k27UsRRjC5g,ASMR Marble run #marblerun #asmr,ASMR Marble run #marblerun #asmr,https://support.google.com/youtube/answer/3037...,"12,387",233.0,3.0,NaN,NaN,2024-10-18 16:55:26.581137
4,English with Jennifer,https://www.youtube.com/watch?v=HNaIXQ_qmrg,Conversation Role Play: The Powers That Be,"In this #short, gain confidence speaking as yo...",https://rr2---sn-uxaxh5ji-aw0s.googlevideo.com...,457,29.0,3.0,NaN,2024-10-16 07,2024-10-18 16:55:26.581137
5,Web Inglés,https://www.youtube.com/watch?v=iOPYuI2dLc0,Intensificadores en Inglés #shorts,Adverbios intensificadores ¿Qué tan cerca está...,https://rr3---sn-5ualdns6.googlevideo.com/vide...,"6,629",564.0,20.0,NaN,2022-12-17 14,2024-10-18 16:55:26.581137
6,ALIF FUN TIME,https://www.youtube.com/watch?v=iI39fwgM0wI,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-18 16:55:26.581137
7,Genetically Modified Skeptic,https://www.youtube.com/watch?v=IzQDQrpPZ38,Gen Z is MORE Muslim than Boomers here?,"Gen Z is more Muslim, Buddhist, and Hindu than...",https://rr5---sn-h0jelne7.googlevideo.com/vide...,"56,781",2400.0,702.0,NaN,2023-12-10 08,2024-10-18 16:55:26.581137


In [133]:
len(loaded_df.columns)
len(data_list)

2

In [134]:
data_list

[('Robert Breaker',
  'Watching Plane Take Off From a Plane #airtravel #airplane #airport t',
  None,
  'https://support.google.com/youtube/answer/3037019#zippy=%2Ccheck-that-youre-signed-into-youtube',
  '6,045',
  253.0,
  16.0,
  [('5 days ago',
    'I never get tired of watching planes, taking off or landing! Still one of man’s most beautiful inventions!'),
   ('5 days ago', 'Thank you for sharing, brother Breaker'),
   ('2 days ago',
    'You one of the things that I have learn is when God gives us eternal life he will never take it back because if he did then it would not be eternal life even if we mess up but God will correct us as his children but he will never take away eternal life from us'),
   ('4 days ago',
    'Just like the airplane is ready to take off it reminds me of the rapture of the church that will take place real soon'),
   ('5 days ago',
    'Ret Captain. If this is thrilling imagine the pilot advancing the throttles releasing the awesome power of large engines!